Données

In [ ]:
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az

# --------------------------
# Synthetic Data Structure
# --------------------------

n_items = 9
n_campaigns = 3
n_aliquots = 3
n_replicates = 2

n_y_per_item = n_campaigns * n_aliquots * n_replicates
n_total_obs = n_items * n_y_per_item

# Create indices
item_idx = np.repeat(np.arange(n_items), n_y_per_item)
campaign_idx = np.tile(np.repeat(np.arange(n_campaigns),
                                 n_aliquots * n_replicates), n_items)
aliquot_idx_within_campaign = np.tile(np.repeat(np.arange(n_aliquots),
                                                n_replicates),
                                       n_items * n_campaigns)
aliquot_global_idx = campaign_idx * n_aliquots
+ aliquot_idx_within_campaign  # unique index per aliquot

# Observed x_i and its uncertainty
# x_obs = np.random.normal(10, 1, size=n_items)
# u_x = np.full(n_items, 0.2)

# Simulated y_obs placeholder
# y_obs = np.random.normal(50, 5, size=n_total_obs)

# Data CCQM-K147
Assigned_Values = pd.DataFrame({
    "X": [4.51, 5.52, 5.91, 8.83, 39.8, 60.6, 65.3, 97, 108],
    "uX": [0.11, 0.13, 0.195, 0.205, 1.35, 0.65, 2.8, 2.00, 5.00]})
x_obs, u_x = Assigned_Values["X"].values,
 Assigned_Values["uX"].values

y_obs = np.array([
    4.449, 4.433, 4.083, 4.554, 4.646, 4.210,   # DMR-486b (campaign 1)
    4.684, 4.806, 4.491, 4.501, 4.486, 4.471,            # (campaign 2)
    4.637, 4.528, 4.692, 4.540, 4.501, 4.620,            # (campaign 3)
    5.433, 5.334, 5.417, 5.459, 5.445, 5.488,   # DMR-274g
    5.598, 5.352, 5.893, 5.692, 5.282, 5.450,
    5.575, 5.436, 5.609, 5.586, 5.189, 5.554,
    6.031, 5.827, 6.024, 6.027, 5.539, 5.721,   # SMR 1549a
    5.592, 5.986, 6.221, 5.981, 5.096, 4.983,
    5.369, 5.394, 5.973, 5.909, 5.556, 5.762,
    8.724, 9.207, 8.916, 8.853, 8.952, 8.636,   # DMR-82c
    9.189, 9.117, 9.179, 8.553, 9.043, 8.979,
    8.609, 8.539, 8.549, 8.421, 8.621, 8.166,
    36.49, 36.94, 37.98, 36.81, 37.50, 37.45,   # GBW10037
    35.01, 35.56, 37.79, 36.14, 35.29, 36.92,
    39.25, 39.49, 37.58, 37.20, 39.14, 37.95,
    60.53, 59.99, 61.79, 61.63, 58.69, 60.85,   # SRM 1869
    61.81, 60.23, 60.11, 61.07, 61.43, 59.44,
    59.16, 59.99, 59.68, 59.58, 60.83, 58.72,
    65.32, 64.56, 65.48, 65.25, 63.64, 64.71,   # SRM 1849a
    61.48, 63.33, 63.02, 63.49, 65.14, 65.15,
    64.66, 64.80, 64.42, 63.86, 63.82, 63.54,
    98.59, 96.57, 96.76, 96.83, 100.47, 100.75, # GBW(E)10027
    98.84, 98.59, 96.48, 94.42, 95.75, 97.97,
    100.27, 96.67, 97.02, 97.21, 100.69, 98.80,
    100.10, 98.51, 99.05, 101.62, 100.24, 100.30,   # 108-02-003
    100.33, 100.29, 101.97, 99.54, 101.29, 101.78,
    99.89, 99.20, 98.11, 98.74, 100.74, 99.48
])

# --------------------------
# PyMC Model
# --------------------------

coords = {
    "item": np.arange(n_items),
    "campaign": np.arange(n_campaigns),
    "aliquot": np.arange(n_campaigns * n_aliquots),
    "obs": np.arange(n_total_obs)
}

In [ ]:
with pm.Model(coords=coords) as model:
    # --- Data ---
    pm.Data("x_obs", x_obs)
    pm.Data("u_x", u_x)
    pm.Data("y_obs", y_obs)
    pm.Data("item_idx", item_idx)
    pm.Data("campaign_idx", campaign_idx)
    pm.Data("aliquot_idx", aliquot_global_idx)

    # --- Priors ---
    a = pm.Normal("a", mu=0, sigma=316)
    b = pm.Normal("b", mu=0, sigma=316)

    sigma_cam = pm.HalfNormal("sigma_cam", sigma=1)
    sigma_ali = pm.HalfNormal("sigma_ali", sigma=1)
    sigma_mat = pm.HalfNormal("sigma_mat", sigma=1)

    # --- Latent true x ---
    x_true = pm.Normal("x_true", mu=x_obs, sigma=u_x, shape=n_items)

    # --- Campaign-level random effects ---
    beta = pm.Normal("beta", mu=0, sigma=sigma_cam, dims="campaign")

    # --- Aliquot-level random effects (nested in campaign) ---
    gamma = pm.Normal("gamma", mu=0, sigma=sigma_ali, dims="aliquot")

    # --- Linear predictor ---
    alpha = (
        a
        + b * x_true[item_idx]xn
        + beta[campaign_idx]
        + gamma[aliquot_global_idx]
    )

    # --- Likelihood ---
    y = pm.Normal("y", mu=alpha, sigma=sigma_mat, observed=y_obs,
                  dims="obs")

    # --- Sampling ---
    trace = pm.sample(draws=4000, tune=4000, target_accept=0.99,
                      nuts_sampler = 'numpyro',
                      return_inferencedata=True, progressbar=True)

    # Summary
    summary = az.summary(trace, hdi_prob = 0.95)
    display(summary)

Sigma = 4 *(Anciennes notations pour la moyenne, la pente et l'ordonnée à l'origine)*

In [ ]:
with pm.Model(coords=coords) as model:
    # --- Data ---
    pm.Data("x_obs", x_obs)
    pm.Data("u_x", u_x)
    pm.Data("y_obs", y_obs)
    pm.Data("item_idx", item_idx)
    pm.Data("campaign_idx", campaign_idx)
    pm.Data("aliquot_idx", aliquot_global_idx)

    # --- Priors ---
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=1, sigma=10)

    sigma_camp = pm.HalfNormal("sigma_camp", sigma=4)
    sigma_aliq = pm.HalfNormal("sigma_aliq", sigma=4)
    sigma_resid = pm.HalfNormal("sigma_resid", sigma=4)

    # --- Latent true x ---
    x_true = pm.Normal("x_true", mu=x_obs, sigma=u_x, shape=n_items)

    # --- Campaign-level random effects ---
    gamma = pm.Normal("gamma", mu=0, sigma=sigma_camp, dims="campaign")

    # --- Aliquot-level random effects (nested in campaign) ---
    delta = pm.Normal("delta", mu=0, sigma=sigma_aliq, dims="aliquot")

    # --- Linear predictor ---
    mu = (
        alpha
        + beta * x_true[item_idx]
        + gamma[campaign_idx]
        + delta[aliquot_global_idx]
    )

    # --- Likelihood ---
    y = pm.Normal("y", mu=mu, sigma=sigma_resid, observed=y_obs,
                  dims="obs")

    # --- Sampling ---
    trace = pm.sample(draws=4000, tune=4000, target_accept=0.99,
                      nuts_sampler = 'numpyro',
                      return_inferencedata=True, progressbar=True)

    summary = az.summary(trace, hdi_prob = 0.95)
    display(summary)

Sigma = 5 *(Anciennes notations pour la moyenne, la pente et l'ordonnée à l'origine)*

In [ ]:
with pm.Model(coords=coords) as model:
    # --- Data ---
    pm.Data("x_obs", x_obs)
    pm.Data("u_x", u_x)
    pm.Data("y_obs", y_obs)
    pm.Data("item_idx", item_idx)
    pm.Data("campaign_idx", campaign_idx)
    pm.Data("aliquot_idx", aliquot_global_idx)

    # --- Priors ---
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=1, sigma=10)

    sigma_camp = pm.HalfNormal("sigma_camp", sigma=5)
    sigma_aliq = pm.HalfNormal("sigma_aliq", sigma=5)
    sigma_resid = pm.HalfNormal("sigma_resid", sigma=5)

    # --- Latent true x ---
    x_true = pm.Normal("x_true", mu=x_obs, sigma=u_x, shape=n_items)

    # --- Campaign-level random effects ---
    gamma = pm.Normal("gamma", mu=0, sigma=sigma_camp, dims="campaign")

    # --- Aliquot-level random effects (nested in campaign) ---
    delta = pm.Normal("delta", mu=0, sigma=sigma_aliq, dims="aliquot")

    # --- Linear predictor ---
    mu = (
        alpha
        + beta * x_true[item_idx]
        + gamma[campaign_idx]
        + delta[aliquot_global_idx]
    )

    # --- Likelihood ---
    y = pm.Normal("y", mu=mu, sigma=sigma_resid, observed=y_obs,
                  dims="obs")

    # --- Sampling ---
    trace = pm.sample(draws=4000, tune=4000, target_accept=0.99,
                      nuts_sampler = 'numpyro',
                      return_inferencedata=True, progressbar=True)

    summary = az.summary(trace, hdi_prob = 0.95)
    display(summary)

Sigma = 6 *(Anciennes notations pour la moyenne, la pente et l'ordonnée à l'origine)*

In [ ]:
with pm.Model(coords=coords) as model:
    # --- Data ---
    pm.Data("x_obs", x_obs)
    pm.Data("u_x", u_x)
    pm.Data("y_obs", y_obs)
    pm.Data("item_idx", item_idx)
    pm.Data("campaign_idx", campaign_idx)
    pm.Data("aliquot_idx", aliquot_global_idx)

    # --- Priors ---
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10)

    sigma_camp = pm.HalfNormal("sigma_camp", sigma=6)
    sigma_aliq = pm.HalfNormal("sigma_aliq", sigma=6)
    sigma_resid = pm.HalfNormal("sigma_resid", sigma=6)

    # --- Latent true x ---
    x_true = pm.Normal("x_true", mu=x_obs, sigma=u_x, shape=n_items)

    # --- Campaign-level random effects ---
    gamma = pm.Normal("gamma", mu=0, sigma=sigma_camp, dims="campaign")

    # --- Aliquot-level random effects (nested in campaign) ---
    delta = pm.Normal("delta", mu=0, sigma=sigma_aliq, dims="aliquot")

    # --- Linear predictor ---
    mu = (
        alpha
        + beta * x_true[item_idx]
        + gamma[campaign_idx]
        + delta[aliquot_global_idx]
    )

    # --- Likelihood ---
    y = pm.Normal("y", mu=mu, sigma=sigma_resid, observed=y_obs,
                  dims="obs")

    # --- Sampling ---
    trace = pm.sample(draws=4000, tune=4000, target_accept=0.99,
                      nuts_sampler = 'numpyro',
                      return_inferencedata=True, progressbar=True)

    summary = az.summary(trace, hdi_prob = 0.95)
    display(summary)

Sigma = 10 *(Anciennes notations pour la moyenne, la pente et l'ordonnée à l'origine)*

In [ ]:
with pm.Model(coords=coords) as model:
    # --- Data ---
    pm.Data("x_obs", x_obs)
    pm.Data("u_x", u_x)
    pm.Data("y_obs", y_obs)
    pm.Data("item_idx", item_idx)
    pm.Data("campaign_idx", campaign_idx)
    pm.Data("aliquot_idx", aliquot_global_idx)

    # --- Priors ---
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10)

    sigma_camp = pm.HalfNormal("sigma_camp", sigma=10)
    sigma_aliq = pm.HalfNormal("sigma_aliq", sigma=10)
    sigma_resid = pm.HalfNormal("sigma_resid", sigma=10)

    # --- Latent true x ---
    x_true = pm.Normal("x_true", mu=x_obs, sigma=u_x, shape=n_items)

    # --- Campaign-level random effects ---
    gamma = pm.Normal("gamma", mu=0, sigma=sigma_camp, dims="campaign")

    # --- Aliquot-level random effects (nested in campaign) ---
    delta = pm.Normal("delta", mu=0, sigma=sigma_aliq, dims="aliquot")

    # --- Linear predictor ---
    mu = (
        alpha
        + beta * x_true[item_idx]
        + gamma[campaign_idx]
        + delta[aliquot_global_idx]
    )

    # --- Likelihood ---
    y = pm.Normal("y", mu=mu, sigma=sigma_resid, observed=y_obs, dims="obs")

    # --- Sampling ---
    trace = pm.sample(draws=4000, tune=4000, target_accept=0.99,
                      nuts_sampler = 'numpyro',
                      return_inferencedata=True, progressbar=True)

    summary = az.summary(trace, hdi_prob = 0.95)
    display(summary)

**Commentaires**

`sigma_resid` : plutôt stable -> bonne information des données ?

`sigma_aliq`: presque nul -> effet faible ?

`sigma_camp` : augmente avec le prior -> mal contraint ?